In [ ]:
# Step 1. Mount drive 
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!nvidia-smi

In [ ]:
# Step 2. Tai ma nguon YOLO ve drive
!rm -rf darknet
%cd /content/gdrive/MyDrive/DetectLight
!git clone https://github.com/AlexeyAB/darknet
%cd /content/gdrive/MyDrive/DetectLight/darknet
!rm -rf data
!mkdir data

In [ ]:
# Step 4. Giải nén file data
%cd /content/gdrive/MyDrive/DetectLight/darknet/data
!unzip data.zip

In [ ]:
# Step 5. Tạo file yolo.names
%cd /content/gdrive/MyDrive/DetectLight/darknet
!echo "Brake Light" > yolo.names
!echo "Left Blinker" >> yolo.names
!echo "Right Blinker" >> yolo.names
!echo "Left Blinker & Brake" >> yolo.names
!echo "Right Blinker & Brake" >> yolo.names
!echo "Auto Brake Light" >> yolo.names
!echo "Auto Left Blinker" >> yolo.names
!echo "Auto Right Blinker" >> yolo.names
!echo "Auto Left Blinker & Brake" >> yolo.names
!echo "Auto Right Blinker & Brake" >> yolo.names

/content/gdrive/MyDrive/DetectLight/darknet


In [ ]:
# Step 6. Tạo file train.txt và val.txt
%cd /content/gdrive/MyDrive/DetectLight/darknet

import glob2
import math  
import os
import numpy as np
 
files = []
for ext in ["*.png", "*.jpeg", "*.jpg"]:
  image_files = glob2.glob(os.path.join("data/data/", ext))
  files += image_files
 
nb_val = math.floor(len(files)*0.2)
rand_idx = np.random.randint(0, len(files), nb_val)
 
# Tạo file train.txt
with open("train.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')
 
# Tạo file val.txt
with open("val.txt", "w") as f:
  for idx in np.arange(len(files)):
    if (idx in rand_idx) and (os.path.exists(files[idx][:-3] + "txt")):
      f.write(files[idx]+'\n')

/content/gdrive/MyDrive/DetectLight/darknet


In [ ]:
# Step 7. Tạo file yolo.data
%cd /content/gdrive/MyDrive/DetectLight/darknet
!mkdir backup
!echo classes=10 > yolo.data
!echo train=train.txt >> yolo.data
!echo valid=val.txt >> yolo.data
!echo names=yolo.names >> yolo.data
!echo backup=backup >> yolo.data

/content/gdrive/MyDrive/DetectLight/darknet
mkdir: cannot create directory ‘backup’: File exists


In [ ]:
# Step 8. Make darknet
%cd /content/gdrive/MyDrive/DetectLight/darknet
!rm darknet
!make

/content/gdrive/MyDrive/DetectLight/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include obj/image_opencv.o obj/http_stream.o obj/gemm.o obj/utils.o obj/dark_cuda.o obj/convolutional_layer.o obj/list.o obj/image.o obj/activations.o obj/im2col.o obj/col2im.o obj/blas.o obj/crop_layer.o obj/dropout_layer.o obj/maxpool_layer.o obj/softmax_layer.o obj/data.o obj/matrix.o obj/network.o obj/connected_layer.o obj/cost_layer.o obj/parser.o obj/option_list.o obj/darknet.o obj/detection_layer.o obj/captcha.o obj/route_layer.o obj/writing.o obj/box.o obj/nightmare.o obj/normalization_layer.o obj/avgpool_layer.o obj/coco.o obj/dice.o obj/yolo.o obj/detector.o obj/layer.o obj/compare.o obj/classifier.o obj/local_layer.o obj/swa

In [ ]:
# Step 9. Download pretrain weight
%cd /content/gdrive/MyDrive/DetectLight/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# Step 10. Train Yolo
%cd /content/gdrive/MyDrive/DetectLight/darknet
!./darknet detector train yolo.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show 

In [ ]:
# Step 11. Train tiếp trên weights đã train
%cd /content/gdrive/MyDrive/DetectLight/darknet
!./darknet detector train yolo.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -dont_show 

In [ ]:
%cd /content/gdrive/MyDrive/DetectLight/darknet
!./darknet detector map yolo.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -iou_thresh 0.5

In [ ]:
%cd /content/gdrive/MyDrive/DetectLight/darknet
!./darknet detector map yolo.data cfg/yolov4-custom.cfg backup/yolov4-custom_last.weights -iou_thresh 0.75